In [ ]:
import tensorflow as tf
from tensorflow.python.client import device_lib
devices = device_lib.list_local_devices()

In [2]:
def sizeof_fmt (num,suffix='B'):
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num)<1024.0:
            return "%3.lf %s%s" % (num,unit,suffix)
        num/=1024.0
    return "%.lf%s%s" % (num,'Yi',suffix)

for d in devices:
    t         = d.device_type
    name      = d.physical_device_desc
    l         = [item.split(':',1) for item in name.split(", ")]
    name_attr = dict([x for x in l if len(x)==2])
    dev       = name_attr.get('name','Unnamed device')
    print(f"{d.name} || {t} || {sizeof_fmt(d.memory_limit)}")

/device:CPU:0 || CPU || 256 MiB
/device:GPU:0 || GPU ||   4 GiB
/device:GPU:1 || GPU ||   9 GiB
/device:GPU:2 || GPU ||   9 GiB


In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

BATCH_SIZE = 32

def process(image,label):
    image = tf.image.resize(image,[299,2999]) /255.0
    return image, label

strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])
print ('Number of devices: %d' % strategy.num_replicas_in_sync)

In [20]:
batch_size = BATCH_SIZE * strategy.num_replicas_in_sync
batch_size

64

In [ ]:
dataset =tfds.loaf("cats_vs_dogs",split=tfds.Split.TRAIN, as_supervised=True)
dataset = dataset.map(process).shuffle(500).batch(batch_size)